In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# !pip install indic-nlp-library

In [8]:
import pandas as pd
import re
from indicnlp.tokenize import indic_tokenize

file_name = '/content/drive/MyDrive/Datasets_HateSpeech/Telugu/Telugu_IMBALANCED.csv'
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_name, encoding='utf-8')
df.head()

,Post,Labels Set,Dataset
0,అనూహ్యంగా పాకి స్థాన్‌ చేతిలో ఓడిన ఇంగ్లండ్‌ వ...,no,Hatespeech_test
1,మరోవైపు డేవిడ్‌ వార్నర్‌ కూడా తన అర్ధసెంచరీ పూ...,no,Hatespeech_test
2,అనూహ్యంగా అశోక్ గ‌జ‌ప‌తిపై గెలుపొందడం రాష్ట్రవ...,no,Hatespeech_test
3,"జిల్లాలోని అమలాపురం, రంపచోడవరం, రామచంద్రపురం డ...",no,Hatespeech_test
4,అగ్ని ప్రమాద సమాచారం అందిన వెంటనే నాలుగు ఫైరిం...,no,Hatespeech_test


In [9]:
# Check for duplicates
duplicate_rows = df[df.duplicated()]
if not duplicate_rows.empty:
    print("Duplicate rows found. Removing duplicates...")
    df.drop_duplicates(inplace=True)

# Check for NaN values
nan_rows = df[df.isna().any(axis=1)]
if not nan_rows.empty:
    print("Rows with NaN values found. Removing...")
    df.dropna(inplace=True)

# Print length of the dataset after removing duplicates and NaN values
print("Length of the dataset after cleaning: ", len(df))

Duplicate rows found. Removing duplicates...
Length of the dataset after cleaning:  35069


In [10]:
def preprocess_telugu_text(text):
    # Remove mentions, email addresses, URLs, RT (retweet)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Remove mentions
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)  # Remove email addresses
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'RT\s*@\S+', '', text)  # Remove retweets
    # Remove HTML tags (if any)
    text = re.sub(r'<.*?>', '', text)
    # Remove English words and non-Telugu characters
    text = re.sub(r'[A-Za-z]', '', text)
    # Remove Telugu punctuation marks
    text = re.sub(r'[\u002C\u0964]', '', text)  # Adjusted regex for punctuation marks
    # Remove special characters and accented characters except numeric characters
    text = re.sub(r'[^\u0C00-\u0C7F0-9\s]', '', text)  # Include numeric characters
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    # Strip leading/trailing whitespaces
    text = text.strip()
    return text


# Preprocess the 'Post' column
df['Processed_Post'] = df['Post'].apply(preprocess_telugu_text)

In [12]:
mapping_dict = {'yes': 1, 'no': 0}
df["Labels Set"] = df["Labels Set"].map(mapping_dict)

In [15]:
set(df['Labels Set'])

{0, 1}

In [19]:
train_hate_df = df[df["Labels Set"] == 1].copy()
train_non_hate_df = df[df["Labels Set"] == 0].copy()

sampled_hate_df = train_hate_df
sampled_non_hate_df = train_non_hate_df.sample(n=4630, random_state=42)

concatenated_df = pd.concat([sampled_hate_df, sampled_non_hate_df], axis=0)
df_ = concatenated_df.sample(frac=1, random_state=42).reset_index(drop=True)

train_df = df_[:4000]
test_df = df_[-1000:]

In [21]:
test_df[test_df['Labels Set'] == 1]

,Post,Labels Set,Dataset,Processed_Post
4006,కోహ్లిని తొందరగా పెవిలియన్‌కు పంపడానికి కవ్విం...,1,Hatespeech_train,కోహ్లిని తొందరగా పెవిలియన్కు పంపడానికి కవ్వింప...
4016,"చంద్రబాబు చేసిన నీచ రాజకీయాలను, అన్యాయాన్ని.\n",1,Hatespeech_train,చంద్రబాబు చేసిన నీచ రాజకీయాలను అన్యాయాన్ని
4033,మహిళలను కించపరిచేలా అభ్యంతరకరంగా మాట్లాడిన పాం...,1,Hatespeech_train,మహిళలను కించపరిచేలా అభ్యంతరకరంగా మాట్లాడిన పాం...
4039,ఓ స్థాయి లో ఉండి కూడా ముఖ్యమంత్రి ఫై అసభ్య కా...,1,Hatespeech_valid,ఓ స్థాయి లో ఉండి కూడా ముఖ్యమంత్రి ఫై అసభ్య కామ...
4041,అయినా జగన్ కు కావల్సింది కాంట్రాక్టులే గాని ప్...,1,Hatespeech_train,అయినా జగన్ కు కావల్సింది కాంట్రాక్టులే గాని ప్...
...,...,...,...,...
4936,కాబట్టి పాక్‌తో మ్యాచ్‌ను నిషేధించే అవకాశం బీస...,1,Hatespeech_train,కాబట్టి పాక్తో మ్యాచ్ను నిషేధించే అవకాశం బీసీస...
4948,ఈ ఘ‌ట‌న ప‌ట్ల టీడీపీ నేత‌లు మండిప‌డుతున్నారు.\n,1,Hatespeech_test,ఈ ఘటన పట్ల టీడీపీ నేతలు మండిపడుతున్నారు
4957,తమ పార్టీ గుర్తుపై గెలిచిన 23 మందిని గత ప్రభుత...,1,Hatespeech_train,తమ పార్టీ గుర్తుపై గెలిచిన 23 మందిని గత ప్రభుత...
4973,పర స్త్రీని మోహించి ఆమె కోసం ఎంతకయినా తెగించే ...,1,Hatespeech_train,పర స్త్రీని మోహించి ఆమె కోసం ఎంతకయినా తెగించే ...
